# Evalution and Analysis

## Imports:

In [1]:
import pandas as pd
from pathlib import Path
import re
import numpy as np
from sklearn.metrics import r2_score, root_mean_squared_error, mean_absolute_error

## Get predictions and metrics:

In [5]:
def get_model_name(fileName:str):
    '''
    This function gets the model name according to the following naming convention: {model name}_{information...}.npy,
    where {information} is optional. 

    Parameters:
    -----------
    fileName: A string of the models predictions file name.

    Returns:
    --------
    A string of the models name.
    '''

    return re.findall(r'([a-zA-Z0-9]+)',fileName)[0]


def get_predictions(predsPath:Path):
    '''
    This function combines all predictions of all models into one dataframe and includes the ground truth.

    Parameters:
    -----------
    predsPath: A path/string of a path, to the directory of all the predictions

    Returns:
    --------
    Returns a pandas Dataframe of all the models predictions, as well as the ground truth. 
    '''
    # Get the ground truth:
    groundTruth= pd.read_csv(Path('../Data/Test/test1990s.csv'),parse_dates=[0],date_format='%m%Y',index_col=0, usecols=[0,1])

    # make an empty Dataframe to store the models predictions (making the index  the observation data):
    predsDf= pd.DataFrame(index=groundTruth.index)

    # Add the ground truth to the predictions dataframe
    predsDf['ground_truth']= groundTruth

    # Loop over all the files in the predictions folder:
    for i in list(predsPath.glob('*.npy')):
        # Add the predictions to the predictions dataframe, where the column is the model name
        predsDf[get_model_name(i.name)]= np.load(i)[:48]
    
    return predsDf

def calc_metrics(predictionsDf:pd.DataFrame, horizon = None):
    '''
    This function calculates the evaluation metrics of each model, given the predictions dataframe.
    The following metrics are used:
        * RMSE
        * MAE
        * r^2 score

    Parameters:
    -----------
    predictionsDf: a pandas datframe containg the ground truth and all the predictions for each model, organized in columns.

    horizon: number of timesteps to calculate the metrics.

    Returns:
    --------
    Returns a pandas Dataframe containg all the evaluation metrics of all the models, where each column represents a metric and each row represents a model.
    '''
    # Deafult to horizon of 2 years:
    if horizon is None:
        horizon= predictionsDf.shape[0]

    # create an empty dataframe with columns reprresnting an evaluation metric
    metricsDf= pd.DataFrame(columns=['RMSE','MAE', 'r2'])
    
    # Loop over all columns/models in the prtedictions dataframe
    for model in predictionsDf.columns.drop('ground_truth'):#
        # Calculate the metrics and add them to the metrics dataframe
        metricsDf.loc[model,'RMSE']=root_mean_squared_error(predictionsDf['ground_truth'].iloc[:horizon],predictionsDf[model].iloc[:horizon])
        metricsDf.loc[model,'MAE']=mean_absolute_error(predictionsDf['ground_truth'].iloc[:horizon],predictionsDf[model].iloc[:horizon])
        metricsDf.loc[model,'r2']=r2_score(predictionsDf['ground_truth'].iloc[:horizon],predictionsDf[model].iloc[:horizon])

    return metricsDf

In [6]:
predsPath= Path('../Predictions/')
predsDf=get_predictions(predsPath)
display(predsDf)

,ground_truth,Naive,SARIMAX
observation_date,,,
01/2021,106.083,105.69,106.037814
02/2021,106.471,105.69,106.500217
03/2021,107.032,105.69,106.874017
04/2021,107.632,105.69,107.081435
05/2021,108.184,105.69,107.416452
06/2021,108.748,105.69,107.783812
07/2021,109.275,105.69,108.098582
08/2021,109.720,105.69,108.374557
09/2021,110.047,105.69,108.517250


In [7]:
metricsDf= calc_metrics(predsDf, horizon=12)
display(metricsDf)

,RMSE,MAE,r2
Naive,3.786199,3.282,-3.022498
SARIMAX,1.452663,1.148779,0.407868


# Analysis: